Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = 'ETH-USD'
#["ETH-USD"]


In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol,end = dt.datetime.now(),start = dt.datetime(2020,11,8))
df = df.iloc[:, :4]
df = df.dropna()
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2020-11-07,454.722565,465.675476,428.456360,435.713135
2020-11-08,435.718811,457.780457,433.153778,453.554779
2020-11-09,453.574158,457.349609,435.163879,444.163055
2020-11-10,444.166382,453.758362,439.600128,449.679626
2020-11-11,449.679657,473.578857,449.524933,462.960541
...,...,...,...,...
2021-11-05,4537.423340,4570.895996,4447.487305,4486.243164
2021-11-06,4482.647949,4530.997070,4334.973633,4521.581055
2021-11-07,4523.981934,4640.921875,4510.984375,4620.554688


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".
df['Close_30_Rolling'] = df['Close'].rolling(window=30).mean()
df['Close_45_Rolling'] = df['Close'].rolling(window=45).mean()
df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1'] = df['Open']-df['Open'].shift(1)
#df['Range']= df['High']-df['Low']
df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_30_Rolling,Close_45_Rolling,Close_15_Rolling,Open_1
Date,,,,,,,,
2020-12-21,638.315186,646.846558,600.836060,609.817871,589.232284,549.267253,600.032853,-20.869873
2020-12-22,609.420532,635.076599,589.552002,634.854187,591.791819,553.692609,602.900240,-28.894653
2020-12-23,634.824585,637.122803,560.364258,583.714600,590.967171,556.585050,604.826029,25.404053
2020-12-24,584.135620,613.815186,568.596375,611.607178,591.224152,560.306030,607.367900,-50.688965
2020-12-25,611.554565,633.061401,605.424438,626.410706,593.081620,564.233388,611.816711,27.418945
...,...,...,...,...,...,...,...,...
2021-11-05,4537.423340,4570.895996,4447.487305,4486.243164,4010.841870,3726.025434,4290.997461,-67.255371
2021-11-06,4482.647949,4530.997070,4334.973633,4521.581055,4041.962077,3758.107948,4327.757406,-54.775391
2021-11-07,4523.981934,4640.921875,4510.984375,4620.554688,4077.188590,3790.664193,4357.683480,41.333984


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(df['Close'].shift(-1) - df['Close'] < 0, -1, 1)
y

array([ 1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,
       -1,  1, -1, -1, -1,  1,  1, -1,  1, -1,  1,  1,  1, -1,  1, -1,  1,
       -1,  1, -1,  1,  1, -1, -1,  1,  1,  1, -1,  1, -1, -1,  1,  1, -1,
        1,  1, -1, -1, -1,  1,  1,  1,  1, -1,  1, -1, -1,  1, -1, -1,  1,
       -1,  1, -1,  1, -1, -1,  1,  1,  1,  1, -1,  1, -1,  1, -1, -1,  1,
        1, -1,  1, -1, -1, -1, -1, -1,  1,  1,  1, -1,  1,  1,  1,  1,  1,
       -1,  1,  1,  1, -1,  1, -1,  1,  1, -1,  1,  1,  1, -1, -1, -1, -1,
        1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1, -1,
       -1,  1,  1,  1,  1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1, -1,
        1,  1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1, -1,  1, -1, -1,  1,
       -1, -1,  1,  1,  1,  1, -1,  1, -1, -1,  1, -1, -1,  1, -1, -1,  1,
        1,  1,  1,  1, -1,  1,  1,  1, -1,  1, -1, -1,  1, -1,  1, -1, -1,
        1, -1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  1,  1, -1,  1

In [6]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train, X_test, y_train , y_test =train_test_split(X, y, test_size= index)
#X_train, X_test, y_train , y_test 

In [7]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

# instantiate the model (using the default parameters)
logistic = LogisticRegression()
#fit the model with data
logistic = logistic.fit(X_train,y_train)
#y_pred=logistic.predict(X_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[0.005472766311578449]
1,High,[-0.0044348251832847]
2,Low,[-0.0020405940423576696]
3,Close,[0.001567228039591155]
4,Close_30_Rolling,[-0.00526048412265182]
5,Close_45_Rolling,[0.003466270592576461]
6,Close_15_Rolling,[0.0014093428228261686]
7,Open_1,[0.001295561246143332]


In [9]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)
#predictions

In [10]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

#Summarize the number of correct and incorrect predictions
metrics.confusion_matrix(y_test,predictions)

array([[34, 65],
       [40, 87]])

In [11]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

accuracy = logistic.score(X_test, y_test)
accuracy

0.5353982300884956

In [12]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

#To Estimating the accuracy of the data

cross_val = cross_val_score(logistic,X_test, y_test)
cross_val

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0.54347826, 0.64444444, 0.53333333, 0.4       , 0.57777778])